# Introduction

This script is a single code snippet to generate level-1 LULC maps of IndiaSAT, Dynamic World, and a combination of both classifiers. It is a GEE implementation.

This script uses Harmonized Sentinel-2 1C to generate LULC maps. It will give maps from June/2015 onwards.

The output classes are-

*   1 : Greenery
*   2 : Water
*   3 : Built-up
*   4 : Barren land

We have also added a confidence score with each prediction.

To incorporate LULC of years further back in time, we can incorporate Landsat-7 and Landsat-8 too.

# Set Coding Environment

## Library Installations

In [ ]:
# !pip install geemap

## Module Imports

In [1]:
# import geemap
import ee
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pprint import pprint

## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/LULC_Experiments_Chahat_Ananjan_Saketh

/content/drive/MyDrive/LULC_Experiments_Chahat_Ananjan_Saketh


## Authenticate to Google Earth Engine

In [4]:
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-Vnl-Z-dMaOuHAQRr3DpUC_ilWgEXpHK6mwXORjvuQE&tc=hO2sQeLCmoOT6nCuHZaKF72-3lgKUpwp68YCU7k_NLc&cc=MEhKCjXFpeaxziA2ExlntzKvowZm1TacY1AiSpPrSSk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmM7-PvsrD4JjlMSrWtXpsDq5KtX9Zf8JcKo2mXur9xbyDkwKKQA8c

Successfully saved authorization token.


In [5]:
ee.Initialize()

# Function Definitions

In [12]:
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask)


def add_normalized_bands(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI') # vegetaion index
  ndwi = image.normalizedDifference(['B8', 'B12']).rename('NDWI') # water index
  ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI') # built-up index

  return image.addBands(ndvi).addBands(ndwi).addBands(ndbi)


def add_all_bands(median_image, min_image, max_image):
  return median_image.select('B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8','B9', 'B10', 'B11','B12','B8A','NDVI','NDWI','NDBI').rename('B1_med','B2_med', 'B3_med', 'B4_med', 'B5_med', 'B6_med', 'B7_med', 'B8_med','B9_med', 'B10_med', 'B11_med','B12_med','B8A_med','NDVI_med','NDWI_med','NDBI_med') \
  .addBands(min_image.select('B2','B3','B4','NDVI','NDWI','NDBI').rename('B2_min','B3_min','B4_min','NDVI_min','NDWI_min','NDBI_min')) \
  .addBands(max_image.select('B2','B3','B4','NDVI','NDWI','NDBI').rename('B2_max','B3_max','B4_max','NDVI_max','NDWI_max','NDBI_max'))


def get_S2_image(startDate, endDate, roi_boundary, bands):
  roi_S2_imageCollection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                             .filterBounds(roi_boundary) \
                             .filterDate(startDate , endDate) \
                             .sort('CLOUD_COVER') \
                             .map(maskS2clouds) \
                             .select(bands) \
                             .map(add_normalized_bands)

  roi_image_median = roi_S2_imageCollection.median()
  roi_image_min = roi_S2_imageCollection.min()
  roi_image_max = roi_S2_imageCollection.max()

  roi_image = add_all_bands(roi_image_median, roi_image_min, roi_image_max)
  return ee.Image(roi_image)

def get_IndiaSAT_LULC(startDate, endDate, roi_boundary, trained_model):
  startDate = datetime.strptime(startDate,"%Y-%m-%d")
  endDate = datetime.strptime(endDate,"%Y-%m-%d")
  midDate = startDate.date() + (endDate-startDate) / 2

  dateList = [startDate, midDate, endDate]

  bands = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8','B9', 'B10', 'B11','B12','B8A']

  classified_image_list = []
  confidence_image_list = []

  for date_i in range(len(dateList)):
    if(date_i == len(dateList)-1):
      curr_startDate = datetime.strftime(dateList[0],"%Y-%m-%d")
      curr_endDate = datetime.strftime(dateList[date_i],"%Y-%m-%d")
    else:
      curr_startDate = datetime.strftime(dateList[date_i],"%Y-%m-%d")
      curr_endDate = datetime.strftime(dateList[date_i+1],"%Y-%m-%d")

    # Read sentinel-2 image for this time frame
    roi_image = get_S2_image(curr_startDate, curr_endDate, roi_boundary, bands)
    roi_image = roi_image.clip(roi_boundary.geometry())

    classified_image = roi_image.classify(trained_model)

    roi_label_image = classified_image.select(['classification']).arrayArgmax().arrayFlatten([['predicted_label']])
    roi_confidence_image = classified_image.select(['classification']).arrayGet(roi_label_image).rename(['confidence'])
    roi_label_image = roi_label_image.add(1).toInt8()

    LULC_image = roi_label_image.addBands(roi_confidence_image)
    classified_image_list.append(LULC_image)

  final_label_image = ee.ImageCollection( classified_image_list ).select(['predicted_label']).mode()

  masked_ic = ee.ImageCollection( classified_image_list ).map( lambda image: image.updateMask( image.select(['predicted_label']).eq(final_label_image) ) )
  final_confidence_image = masked_ic.select(['confidence']).max()

  indiasat_ROI_LULC_image = final_label_image.addBands(final_confidence_image)

  return indiasat_ROI_LULC_image



def get_DW_LULC(startDate, endDate, roi_boundary):
  DW_ImageCollection = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
                        .filterDate(startDate, endDate) \
                        .filterBounds(roi_boundary)

  DW_confidence_image = DW_ImageCollection.select(['water','trees','grass','flooded_vegetation','crops','shrub_and_scrub','built','bare','snow_and_ice']).mean().toArray()
  DW_label_image = DW_confidence_image.arrayArgmax().arrayFlatten([['predicted_label']])
  DW_confidence_image = DW_confidence_image.arrayGet(DW_label_image).rename(['confidence'])
  DW_label_image = DW_label_image.add(1).toInt8()

  DW_ROI_LULC_image = DW_label_image.addBands(DW_confidence_image)
  DW_ROI_LULC_image = DW_ROI_LULC_image.clip(roi_boundary.geometry())

  return DW_ROI_LULC_image


def combine_IndiaSAT_DW(indiasat_ROI_LULC_image, DW_ROI_LULC_image):
  # keeping indiasat labels in rows and DW labels in column. 0 index represents masked out pixel (like background).
  # IndiaSAT_DW_mapper = [
  #                       [0,2,1,1,4,1,4,3,4,2],
  #                       [1,1,1,1,4,1,1,3,4,1],
  #                       [2,2,1,2,2,1,4,3,4,2],
  #                       [3,3,1,3,4,1,4,3,4,4],
  #                       [4,4,4,1,4,1,4,3,4,4]
  #                     ]
  IndiaSAT_DW_mapper = [
                      [0,2,1,1,4,1,4,3,4,2],
                      [1,1,1,1,4,1,1,3,4,1],
                      [2,2,1,2,2,1,1,3,4,2],
                      [3,3,1,3,4,1,4,3,4,4],
                      [4,4,1,1,4,1,1,3,4,4]
                    ]

  # IndiaSAT_DW_prob_mapper = [
  #                           [0,1,1,1,1,1,1,1,1,1],
  #                           [0,0,0,0,1,0,0,1,1,0],
  #                           [0,0,1,0,0,1,1,1,1,0],
  #                           [0,0,1,0,1,1,1,0,1,1],
  #                           [0,0,0,1,0,1,0,1,0,0]
  #                           ]

  IndiaSAT_DW_prob_mapper = [
                          [0,1,1,1,1,1,1,1,1,1],
                          [0,0,0,0,1,0,0,1,1,0],
                          [0,0,1,0,0,1,1,1,1,0],
                          [0,0,1,0,1,1,1,0,1,1],
                          [0,0,1,1,0,1,1,1,0,0]
                          ]
  combined_output_image = indiasat_ROI_LULC_image.select(['predicted_label'])
  combined_confidence_image = indiasat_ROI_LULC_image.select(['confidence'])

  indiaSAT_labels = [0,1,2,3,4]
  DW_labels = [0,1,2,3,4,5,6,7,8,9];

  for i in indiaSAT_labels:
    for j in DW_labels:
      combined_output_image = combined_output_image.where(
                                indiasat_ROI_LULC_image.select('predicted_label').eq(i)
                                  .And(
                                    DW_ROI_LULC_image.select('predicted_label').eq(j)
                                  ),
                                IndiaSAT_DW_mapper[i][j] )

      if IndiaSAT_DW_prob_mapper[i][j] == 1:
        combined_confidence_image = combined_confidence_image.where(
                              indiasat_ROI_LULC_image.select('predicted_label').eq(i)
                              .And(
                                DW_ROI_LULC_image.select('predicted_label').eq(j)
                              ),
                              DW_ROI_LULC_image.select('confidence')
                            )

  # print(combined_output_image.bandNames())
  result = combined_output_image.select('predicted_label').rename('Predicted_L1_Label').addBands( combined_confidence_image.select(['confidence']).rename(['confidence_L1']) )
  return result


# Take User Inputs

In [13]:
# example- projects/ee-indiasat/assets/IndiaSat_test_polygons
# example- projects/ee-indiasat/assets/India_Boundary
# example- projects/ee-indiasat/assets/india_district_boundaries
# example- projects/ee-indiasat/assets/India_state_boundaries
# example- projects/ee-indiasat/assets/mandya_jaltol_boundary
# example- projects/ee-ananjan/assets/Wassan
roi_shapefile_path = input("\n Enter the shapefile path containing your Region Of Interest (ROI): \t")

choice = input("\n Do you want to enter name of your ROI to filter within the shapefile?  \n 1. Yes \n 2. No \n")

if choice == '1':
  roi_name = input("\n Enter the name of your ROI : \t")
  filename_prefix = roi_name
else:
  roi_name = ''
  filename_prefix = input('\nEnter the output filename prefix of your prediction maps (areaName): \t')

# Read the shapefile as feature collection
if roi_name == '':
  roi_boundary = ee.FeatureCollection(roi_shapefile_path)
else:
  roi_boundary = ee.FeatureCollection(roi_shapefile_path).filter(ee.Filter.eq('Name',roi_name))

# pprint(roi_boundary.getInfo())


 Enter the shapefile path containing your Region Of Interest (ROI): 	projects/ee-indiasat/assets/india_district_boundaries

 Do you want to enter name of your ROI to filter within the shapefile?  
 1. Yes 
 2. No 
1

 Enter the name of your ROI : 	Baran


In [8]:
# roi_boundary = ee.FeatureCollection('users/mtpictd/agro_eco_regions').filter(ee.Filter.eq('ae_regcode',13))
# filename_prefix = 'EcoRegion13'
print(roi_boundary.get('Name'))

ee.ComputedObject({
  "functionInvocationValue": {
    "functionName": "Element.get",
    "arguments": {
      "object": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.loadTable",
                "arguments": {
                  "tableId": {
                    "constantValue": "4/1AfJohXmM7-PvsrD4JjlMSrWtXpsDq5KtX9Zf8JcKo2mXur9xbyDkwKKQA8c"
                  }
                }
              }
            },
            "filter": {
              "functionInvocationValue": {
                "functionName": "Filter.equals",
                "arguments": {
                  "leftField": {
                    "constantValue": "Name"
                  },
                  "rightValue": {
                    "constantValue": "Baran"
                  }
                }
              }
            }
          }
    

# Execute Code

In [14]:
'''
TRAINING CODE
'''
# Loading the training dataset
# training_data = ee.FeatureCollection('projects/ee-indiasat/assets/IndiaSat_balanced_training_data_Sentinel2')

training_data = ee.FeatureCollection('projects/ee-indiasat/assets/Rasterized_Groundtruth/IndiaSat_balanced_training_data_Sentinel2_2018_19')

band_names = ['B1_med','B2_med', 'B3_med', 'B4_med', 'B5_med', 'B6_med', 'B7_med', 'B8_med','B9_med', 'B10_med', 'B11_med','B12_med','B8A_med','NDVI_med','NDWI_med','NDBI_med',
              'B2_min','B3_min','B4_min','NDVI_min','NDWI_min','NDBI_min',
              'B2_max','B3_max','B4_max','NDVI_max','NDWI_max','NDBI_max']


trained_model = ee.Classifier.smileRandomForest(numberOfTrees=100, seed=42).setOutputMode('MULTIPROBABILITY').train(
                            features = training_data,
                            classProperty = 'class',
                            inputProperties = band_names )

# pprint(trained_model.getInfo())


'''
INFERENCE CODE
'''

startDate = '2015-07-01'
endDate = '2022-07-01'

loopStart = startDate
loopEnd = (datetime.strptime(endDate,"%Y-%m-%d")+relativedelta(years=1)).strftime("%Y-%m-%d")

while loopStart != loopEnd:
  currStartDate = datetime.strptime(loopStart,"%Y-%m-%d")
  currEndDate = (currStartDate+relativedelta(years=1)-timedelta(days=1)).strftime("%Y-%m-%d")
  loopStart = (currStartDate+relativedelta(years=1)).strftime("%Y-%m-%d")

  currStartDate = currStartDate.strftime("%Y-%m-%d")

  print("\n EXECUTING LULC PREDICTION FOR ",currStartDate," TO ",currEndDate,"\n")

  curr_filename = filename_prefix + '_' + currStartDate + "_" + currEndDate

  indiasat_ROI_LULC_image = get_IndiaSAT_LULC(currStartDate, currEndDate, roi_boundary, trained_model)
  DW_ROI_LULC_image = get_DW_LULC(currStartDate, currEndDate, roi_boundary)
  combined_output_image = combine_IndiaSAT_DW(indiasat_ROI_LULC_image, DW_ROI_LULC_image)

  scale = 30
  final_output_filename = curr_filename+'_Level1_LULCmap_'+str(scale)+'m'
  final_output_assetid = 'projects/ee-indiasat/assets/LULC_Deliverables_WithConfidence/Modified/' + final_output_filename

  # Setup the task
  image_export_task3 = ee.batch.Export.image.toAsset(
    image = combined_output_image.clip(roi_boundary.geometry()),
    description = final_output_filename,
    assetId = final_output_assetid,
    scale = scale,
    pyramidingPolicy = {'Predicted_L1_Label': 'mode'},
    maxPixels = 1e13,
    crs = 'EPSG:4326'
  )

  image_export_task3.start()




 EXECUTING LULC PREDICTION FOR  2015-07-01  TO  2016-06-30 


 EXECUTING LULC PREDICTION FOR  2016-07-01  TO  2017-06-30 


 EXECUTING LULC PREDICTION FOR  2017-07-01  TO  2018-06-30 


 EXECUTING LULC PREDICTION FOR  2018-07-01  TO  2019-06-30 


 EXECUTING LULC PREDICTION FOR  2019-07-01  TO  2020-06-30 


 EXECUTING LULC PREDICTION FOR  2020-07-01  TO  2021-06-30 


 EXECUTING LULC PREDICTION FOR  2021-07-01  TO  2022-06-30 


 EXECUTING LULC PREDICTION FOR  2022-07-01  TO  2023-06-30 

